In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv, description="Query arxiv papers")
print(arxiv.name)

arxiv


In [4]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [6]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [8]:
from dotenv import load_dotenv
load_dotenv()

import os

tavily_api_key = os.getenv("TAVILY_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults(tavily_api_key = tavily_api_key)

In [10]:
tavily.invoke("Provide me the latest AI news")

[{'title': 'AI News | Latest AI News, Analysis & Events',
  'url': 'https://www.artificialintelligence-news.com/',
  'content': 'AI News reports on the latest artificial intelligence news and insights. Explore industry trends from the frontline of AI.',
  'score': 0.71185225},
 {'title': 'AI News - YouTube',
  'url': 'https://www.youtube.com/@AINewsOfficial',
  'content': 'AI News delivers the latest breakthroughs in artificial intelligence, machine learning, deep learning, brain computer interface, robotics, and other future',
  'score': 0.6672566},
 {'title': 'Artificial intelligence - NBC News',
  'url': 'https://www.nbcnews.com/artificial-intelligence',
  'content': "![](https://media-cldnry.s-nbcnews.com/image/upload/t_focal-260x130,f_auto,q_auto:best/mpx/2704722219/2025_03/1741305825834_nn_emc_aicompanions_250306_1920x1080-uqwbkq.jpg)\n\n## 02:54\n\n### [A.I. chatbots becoming popular but worry privacy advocates](https://www.nbcnews.com/nightly-news/video/a-i-chatbots-becoming-po

In [11]:
# combine all these tools in the list

tools = [arxiv, wiki, tavily]

In [12]:
# initialize the LLM model

from langchain_groq import ChatGroq

llm = ChatGroq(model = "qwen-qwq-32b")

In [13]:
llm.invoke("What is AI?")

AIMessage(content='\n<think>\nOkay, the user is asking, "What is AI?" Let me start by breaking down the term. AI stands for Artificial Intelligence. I need to explain what that means in simple terms. Maybe start with a basic definition: the simulation of human intelligence in machines. But wait, that\'s a bit vague. I should mention how it involves tasks like learning, problem-solving, and decision-making. \n\nHmm, I should differentiate between narrow or weak AI and general or strong AI. Weak AI is what\'s used now, like voice assistants or recommendation systems. Strong AI would have human-level consciousness, which doesn\'t exist yet. That\'s an important distinction to avoid confusion. \n\nI should also touch on the main fields within AI, like machine learning, neural networks, natural language processing. Maybe give some examples people are familiar with, like Siri, Alexa, self-driving cars. That makes it relatable. \n\nWait, the user might not know the difference between AI and m

In [14]:
llm_with_tools = llm.bind_tools(tools = tools)

In [15]:
llm_with_tools.invoke("what is the recent news on AI?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_chkv', 'function': {'arguments': '{"query":"recent news on artificial intelligence"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 193, 'prompt_tokens': 356, 'total_tokens': 549, 'completion_time': 0.452069563, 'prompt_time': 0.020308432, 'queue_time': 0.253835761, 'total_time': 0.472377995}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_a91d9c2cfb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3fcd6616-6bb4-4c80-855e-2eb4dd70a419-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'recent news on artificial intelligence'}, 'id': 'call_chkv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 356, 'output_tokens': 193, 'total_tokens': 549})

In [16]:
llm_with_tools.invoke("What is the latest research on quantum computing?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_d74z', 'function': {'arguments': '{"query":"latest research quantum computing"}', 'name': 'arxiv'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 229, 'prompt_tokens': 357, 'total_tokens': 586, 'completion_time': 0.522786107, 'prompt_time': 0.042990482, 'queue_time': 0.769766552, 'total_time': 0.565776589}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_28178d7ff6', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9ba10bc7-c364-493f-9100-9d4a7dbf0efa-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'latest research quantum computing'}, 'id': 'call_d74z', 'type': 'tool_call'}], usage_metadata={'input_tokens': 357, 'output_tokens': 229, 'total_tokens': 586})

In [17]:
llm_with_tools.invoke("What is machine learning?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1dff', 'function': {'arguments': '{"query":"machine learning"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 228, 'prompt_tokens': 353, 'total_tokens': 581, 'completion_time': 0.520385258, 'prompt_time': 0.035003872, 'queue_time': 0.257973597, 'total_time': 0.55538913}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_a91d9c2cfb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ae2c4f5c-bedf-4ae8-bdf0-7a814d76cc9b-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'machine learning'}, 'id': 'call_1dff', 'type': 'tool_call'}], usage_metadata={'input_tokens': 353, 'output_tokens': 228, 'total_tokens': 581})

In [ ]:
# Note the 'name' key in all the AIMessage. The LLM will know which tool to call based on the input provided by the user.